In [1]:
import streamlit as st
import pandas as pd
import fmpsdk
from IPython.display import display
import time

In [2]:
API_KEY = "ePuHa7Ugbp1yWxJrDEgPwolHArwIMHhB"

HERE WE ARE GOING TO RANK THE STOCKS BASED ON ROIC AND EPS FOR EACH YEAR

In [3]:
financial_data=pd.read_csv("symbols_roic_eps.csv")
financial_data = financial_data.dropna()


In [4]:
# Convert 'calendarYear' column to integer
financial_data["calendarYear"] = financial_data["calendarYear"].astype(int)


In [5]:
financial_data.head(100)

,Unnamed: 0,symbol,date,calendarYear,period,roic,earningsYield
0,0,AAPL,2024-09-28,2024,FY,0.443071,0.026819
1,1,AAPL,2023-09-30,2023,FY,0.433892,0.035983
2,2,AAPL,2022-09-24,2022,FY,0.451748,0.040913
3,3,AAPL,2021-09-25,2021,FY,0.389251,0.038586
4,4,AAPL,2020-09-26,2020,FY,0.242695,0.029467
...,...,...,...,...,...,...,...
95,95,TSM,2019-12-31,2019,FY,0.180743,0.040227
96,96,TSM,2018-12-31,2018,FY,0.180925,0.061692
97,97,TSM,2017-12-31,2017,FY,0.190271,0.056912
98,98,TSM,2016-12-31,2016,FY,-0.196726,0.071020


In [6]:
financial_data.loc[financial_data["calendarYear"]==2024]

,Unnamed: 0,symbol,date,calendarYear,period,roic,earningsYield
0,0,AAPL,2024-09-28,2024,FY,0.443071,0.026819
10,10,MSFT,2024-06-30,2024,FY,0.222731,0.025968
20,20,NVDA,2024-01-28,2024,FY,0.512868,0.019295
30,30,AMZN,2024-12-31,2024,FY,0.174774,0.025786
40,40,GOOG,2024-12-31,2024,FY,0.260052,0.042491
...,...,...,...,...,...,...,...
22266,22266,SONO,2024-09-28,2024,FY,-0.121035,-0.025480
22301,22301,GTY,2024-12-31,2024,FY,0.500709,0.043432
22332,22332,LKFN,2024-12-31,2024,FY,0.000000,0.052946
22382,22382,PHR,2024-01-31,2024,FY,-0.499802,-0.098462


In [7]:
# Function to apply ranking within each calendar year
def rank_within_year(group):
    group["roic_rank"] = group["roic"].rank(ascending=False, method="dense")
    group["earnings_yield_rank"] = group["earningsYield"].rank(ascending=False, method="dense")
    
    # Fill NaN values with -1 (or 0) before converting to int
    group["roic_rank"] = group["roic_rank"].fillna(-1).astype(int)
    group["earnings_yield_rank"] = group["earnings_yield_rank"].fillna(-1).astype(int)
    # Calculate f_rank (sum of rankings)
    group["f_rank"] = group["roic_rank"] + group["earnings_yield_rank"]

    # **Re-rank f_rank to start at 1 within each calendar year**
    group["f_rank"] = group["f_rank"].rank(method="dense", ascending=True).astype(int)

    
    return group

# Apply ranking within each `calendarYear`
financial_data = financial_data.groupby("calendarYear", group_keys=False).apply(rank_within_year)





C:\Users\PP124ZJ\AppData\Local\Temp\ipykernel_22300\4189647929.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  financial_data = financial_data.groupby("calendarYear", group_keys=False).apply(rank_within_year)


In [ ]:
financial_data = financial_data.sort_values(by=["calendarYear", "f_rank"], ascending=[False, True])
display(financial_data.loc[financial_data["calendarYear"]==2024].head(20))

,Unnamed: 0,symbol,date,calendarYear,period,roic,earningsYield,roic_rank,earnings_yield_rank,f_rank
3339,3339,PRH,2024-12-31,2024,FY,5.802831,0.304046,4,10,1
1527,1527,MO,2024-12-31,2024,FY,0.350172,0.125387,36,42,2
21042,21042,TSLX,2024-12-31,2024,FY,108.823162,0.095170,1,86,3
14010,14010,LEA,2024-12-31,2024,FY,10.962002,0.094682,2,89,4
5425,5425,BLL,2024-12-31,2024,FY,0.245121,0.238005,78,14,5
7015,7015,BALL,2024-12-31,2024,FY,0.245121,0.238005,78,14,5
13028,13028,CROX,2024-12-31,2024,FY,0.257511,0.146075,68,30,6
14833,14833,AB,2024-12-31,2024,FY,0.220598,0.277375,92,12,7
13038,13038,MGRB,2024-12-31,2024,FY,0.241291,0.128777,79,38,8
13197,13197,MGR,2024-12-31,2024,FY,0.241291,0.128777,79,38,8


In [9]:
display(financial_data.loc[financial_data["calendarYear"]==2015].head(20))

,Unnamed: 0,symbol,date,calendarYear,period,roic,earningsYield,roic_rank,earnings_yield_rank,f_rank
12237,12237,MTG,2015-12-31,2015,FY,0.386699,0.390895,24,10,1
9748,9748,HLNE,2015-03-31,2015,FY,0.352893,0.232443,31,25,2
3765,3765,UAL,2015-12-31,2015,FY,0.301535,0.340685,47,12,3
5573,5573,IHG,2015-12-31,2015,FY,0.459645,0.175186,16,48,4
8673,8673,AAL,2015-12-31,2015,FY,0.276126,0.268843,54,21,5
11554,11554,RMBS,2015-12-31,2015,FY,0.362673,0.158855,29,62,6
1104,1104,GILD,2015-12-31,2015,FY,0.432155,0.122234,19,100,7
14640,14640,LAZ,2015-12-31,2015,FY,0.249474,0.174804,74,49,8
13419,13419,DNB,2015-12-31,2015,FY,0.214041,0.223136,105,26,9
7534,7534,UNM,2015-12-31,2015,FY,2.175292,0.105453,3,138,10


HERE WE ARE GOING TO CALCULATE THE YIELD RATE IF WE BUY THE 20 BEST RANKED STOCKS AT THE BEGINNING OF EACH YEAR AND SELL AT THE BEGINNING OF NEXT YEAR

In [ ]:
'''from datetime import date, timedelta

def second_monday(year):
    # Start from the first day of the year
    first_day = date(year, 1, 1)

    # Find the first Monday
    first_monday = first_day + timedelta(days=(7 - first_day.weekday()) % 7)

    # Calculate the second Monday
    second_monday = first_monday + timedelta(weeks=1)

    # Return the formatted date
    return second_monday.strftime("%Y-%m-%d")

# Example: Get the second Monday of 2024
print(second_monday(2016))'''

2016-01-11


In [39]:
financial_data_top_20=financial_data.loc[financial_data["f_rank"]<26]

In [40]:
pd.DataFrame(fmpsdk.historical_price_full(apikey=API_KEY, symbol="CNA", from_date="2023-12-29", to_date="2023-12-29"))

,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime
0,2023-12-29,42.39,42.69,42.02,42.31,39.02,154533,154533,-0.08,-0.18872,42.3525,"December 29, 23",-0.001887


In [41]:
import datetime

# List of known holidays (extend as needed)
HOLIDAYS = [
  "2025-01-01", "2025-01-20", "2025-02-17", "2025-04-18", "2025-05-26", "2025-06-19", "2025-07-04", "2025-09-01", "2025-11-27", "2025-12-25",
  "2024-01-01", "2024-01-15", "2024-02-19", "2024-03-29", "2024-05-27", "2024-06-19", "2024-07-04", "2024-09-02", "2024-11-28", "2024-12-25",
  "2023-01-02", "2023-01-16", "2023-02-20", "2023-04-07", "2023-05-29", "2023-06-19", "2023-07-04", "2023-09-04", "2023-11-23", "2023-12-25",
  "2022-01-17", "2022-02-21", "2022-04-15", "2022-05-30", "2022-06-20", "2022-07-04", "2022-09-05", "2022-11-24", "2022-12-26",
  "2021-01-01", "2021-01-18", "2021-02-15", "2021-04-02", "2021-05-31", "2021-07-05", "2021-09-06", "2021-11-25", "2021-12-24",
  "2020-01-01", "2020-01-20", "2020-02-17", "2020-04-10", "2020-05-25", "2020-07-03", "2020-09-07", "2020-11-26", "2020-12-25",
  "2019-01-01", "2019-01-21", "2019-02-18", "2019-04-19", "2019-05-27", "2019-07-04", "2019-09-02", "2019-11-28", "2019-12-25",
  "2018-01-01", "2018-01-15", "2018-02-19", "2018-03-30", "2018-05-28", "2018-07-04", "2018-09-03", "2018-11-22", "2018-12-25",
  "2017-01-02", "2017-01-16", "2017-02-20", "2017-04-14", "2017-05-29", "2017-07-04", "2017-09-04", "2017-11-23", "2017-12-25",
  "2016-01-01", "2016-01-18", "2016-02-15", "2016-03-25", "2016-05-30", "2016-07-04", "2016-09-05", "2016-11-24", "2016-12-26",
  "2015-01-01", "2015-01-19", "2015-02-16", "2015-04-03", "2015-05-25", "2015-07-03", "2015-09-07", "2015-11-26", "2015-12-25",
  "2014-01-01", "2014-01-20", "2014-02-17", "2014-04-18", "2014-05-26", "2014-07-04", "2014-09-01", "2014-11-27", "2014-12-25",
  "2013-01-01", "2013-01-21", "2013-02-18", "2013-03-29", "2013-05-27", "2013-07-04", "2013-09-02", "2013-11-28", "2013-12-25",
  "2012-01-02", "2012-01-16", "2012-02-20", "2012-04-06", "2012-05-28", "2012-07-04", "2012-09-03", "2012-11-22", "2012-12-25",
  "2011-01-17", "2011-02-21", "2011-04-22", "2011-05-30", "2011-07-04", "2011-09-05", "2011-11-24", "2011-12-26",
  "2010-01-01", "2010-01-18", "2010-02-15", "2010-04-02", "2010-05-31", "2010-07-05", "2010-09-06", "2010-11-25", "2010-12-24"
]


# Function to get the next workday, skipping weekends and holidays
def get_next_workday(date):
    # Convert string date to datetime object
    date = datetime.datetime.strptime(date, "%Y-%m-%d")

    # Move forward if it's a weekend or holiday
    while date.weekday() in [5, 6] or date.strftime("%Y-%m-%d") in HOLIDAYS:
        date += datetime.timedelta(days=1)

    return date.strftime("%Y-%m-%d")  # Convert back to string

# Filter out 2024 data
financial_data_top_20 = financial_data_top_20[financial_data_top_20["date"].str[:4] != "2024"]

# Initialize an empty list to store closing prices
close_prices = []

# Loop through each row in financial_data to fetch the closing price
for index, row in financial_data_top_20.iterrows():
    symbol = row["symbol"]
    date = row["date"]  # Ensure date format is YYYY-MM-DD
    
    # Get the adjusted workday for the original date
    adjusted_date = get_next_workday(date)
    
    # Get the adjusted workday for the same date one year later
    date_one_year_later = (datetime.datetime.strptime(date, "%Y-%m-%d") + datetime.timedelta(days=365)).strftime("%Y-%m-%d")
    adjusted_date_one_year_later = get_next_workday(date_one_year_later)
    
    print(f"Fetching data for {symbol} on {adjusted_date} and {adjusted_date_one_year_later}")

    # Fetch historical price data for the original date
    try:
        price_data = fmpsdk.historical_price_full(apikey=API_KEY, symbol=symbol, from_date=adjusted_date, to_date=adjusted_date)
        
        if isinstance(price_data, list) and len(price_data) > 0:
            price_df = pd.DataFrame(price_data)  # Directly convert the list into a DataFrame
            close_price = price_df["close"].iloc[0]  # Get the closing price
        else:
            close_price = None  # No valid data found

    except Exception as e:
        print(f"Error fetching data for {symbol} on {adjusted_date}: {e}")
        close_price = None

    # Fetch historical price data for one year later
    try:
        price_data_one_year_later = fmpsdk.historical_price_full(apikey=API_KEY, symbol=symbol, from_date=adjusted_date_one_year_later, to_date=adjusted_date_one_year_later)
        
        if isinstance(price_data_one_year_later, list) and len(price_data_one_year_later) > 0:
            price_df_one_year_later = pd.DataFrame(price_data_one_year_later)
            close_price_one_year_later = price_df_one_year_later["close"].iloc[0]
        else:
            close_price_one_year_later = None

    except Exception as e:
        print(f"Error fetching data for {symbol} on {adjusted_date_one_year_later}: {e}")
        close_price_one_year_later = None

    # Append results to list
    close_prices.append({
        "symbol": symbol,
        "date": date,
        "adjusted_date": adjusted_date,
        "close": close_price,
        "adjusted_date_one_year_later": adjusted_date_one_year_later,
        "close_one_year_later": close_price_one_year_later
    })

# Convert list to DataFrame
close_df = pd.DataFrame(close_prices)

# Merge the closing prices into financial_data
financial_data_top_20 = financial_data_top_20.merge(close_df, on=["symbol", "date"], how="left")

# Display the final DataFrame in Streamlit
display(financial_data_top_20)


Fetching data for GPOR on 2024-01-02 and 2024-12-30
Fetching data for FIHL on 2024-01-02 and 2024-12-30
Fetching data for CALM on 2023-06-05 and 2024-06-03
Fetching data for APA on 2024-01-02 and 2024-12-30
Fetching data for FINV on 2024-01-02 and 2024-12-30
Fetching data for SPNT on 2024-01-02 and 2024-12-30
Fetching data for LNG on 2024-01-02 and 2024-12-30
Fetching data for CEIX on 2024-01-02 and 2024-12-30
Fetching data for NOG on 2024-01-02 and 2024-12-30
Fetching data for CVI on 2024-01-02 and 2024-12-30
Fetching data for ARLP on 2024-01-02 and 2024-12-30
Fetching data for TRMD on 2024-01-02 and 2024-12-30
Fetching data for INSW on 2024-01-02 and 2024-12-30
Fetching data for FMC on 2024-01-02 and 2024-12-30
Fetching data for AMR on 2024-01-02 and 2024-12-30
Fetching data for BWLP on 2024-01-02 and 2024-12-30
Fetching data for OGN on 2024-01-02 and 2024-12-30
Fetching data for PBF on 2024-01-02 and 2024-12-30
Fetching data for HAFN on 2024-01-02 and 2024-12-30


Fetching data for AGO on 2024-01-02 and 2024-12-30
Fetching data for HG on 2024-01-02 and 2024-12-30
Fetching data for CPE on 2024-01-02 and 2024-12-30


Fetching data for ERF on 2024-01-02 and 2024-12-30


Fetching data for ADX on 2024-01-02 and 2024-12-30
Fetching data for LBRT on 2024-01-02 and 2024-12-30
Fetching data for VC on 2024-01-02 and 2024-12-30
Fetching data for AMR on 2023-01-03 and 2024-01-02
Fetching data for ARCH on 2023-01-03 and 2024-01-02
Fetching data for ZIM on 2023-01-03 and 2024-01-02
Fetching data for GPOR on 2023-01-03 and 2024-01-02
Fetching data for PBF on 2023-01-03 and 2024-01-02
Fetching data for EXE on 2023-01-03 and 2024-01-02
Fetching data for CHK on 2023-01-03 and 2024-01-02
Fetching data for SWN on 2023-01-03 and 2024-01-02
Fetching data for GTX on 2023-01-03 and 2024-01-02
Fetching data for INGM on 2023-01-03 and 2024-01-02


Fetching data for AEL on 2023-01-03 and 2024-01-02
Fetching data for CRK on 2023-01-03 and 2024-01-02
Fetching data for HCC on 2023-01-03 and 2024-01-02
Fetching data for BNTX on 2023-01-03 and 2024-01-02
Fetching data for ATKR on 2022-09-30 and 2023-10-02
Fetching data for CPE on 2023-01-03 and 2024-01-02
Fetching data for WIRE on 2023-01-03 and 2024-01-02
Fetching data for ESTE on 2023-01-03 and 2024-01-02


...

Fetching data for APLD on 2005-05-31 and 2006-05-31


Fetching data for FA on 2004-12-31 and 2006-01-02


Fetching data for APLD on 2004-05-31 and 2005-05-31


Fetching data for RXO on 2001-12-31 and 2002-12-31


,Unnamed: 0,symbol,date,calendarYear,period,roic,earningsYield,roic_rank,earnings_yield_rank,f_rank,adjusted_date,close,adjusted_date_one_year_later,close_one_year_later
0,17365,GPOR,2023-12-31,2023,FY,0.514111,0.592272,16,4,1,2024-01-02,134.09,2024-12-30,181.50
1,21373,FIHL,2023-12-31,2023,FY,0.425341,1.472357,22,2,2,2024-01-02,12.33,2024-12-30,17.73
2,13765,CALM,2023-06-03,2023,FY,0.413915,0.327624,26,18,3,2023-06-05,47.09,2024-06-03,61.12
3,10703,APA,2023-12-31,2023,FY,0.447409,0.258347,19,34,4,2024-01-02,36.11,2024-12-30,22.42
4,21142,FINV,2023-12-31,2023,FY,0.333614,0.240103,48,43,5,2024-01-02,4.82,2024-12-30,6.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,17303,FA,2005-12-31,2005,FY,0.063690,0.056081,1,2,1,2006-01-02,NaN,2007-01-01,NaN
354,22032,APLD,2005-05-31,2005,FY,-0.194884,-0.212506,3,3,2,2005-05-31,NaN,2006-05-31,NaN
355,17304,FA,2004-12-31,2004,FY,0.032673,0.010907,1,1,1,2004-12-31,NaN,2006-01-02,NaN
356,22033,APLD,2004-05-31,2004,FY,-0.046739,-0.084726,2,2,2,2004-05-31,NaN,2005-05-31,NaN


In [42]:
close_df

,symbol,date,adjusted_date,close,adjusted_date_one_year_later,close_one_year_later
0,GPOR,2023-12-31,2024-01-02,134.09,2024-12-30,181.50
1,FIHL,2023-12-31,2024-01-02,12.33,2024-12-30,17.73
2,CALM,2023-06-03,2023-06-05,47.09,2024-06-03,61.12
3,APA,2023-12-31,2024-01-02,36.11,2024-12-30,22.42
4,FINV,2023-12-31,2024-01-02,4.82,2024-12-30,6.80
...,...,...,...,...,...,...
353,FA,2005-12-31,2006-01-02,NaN,2007-01-01,NaN
354,APLD,2005-05-31,2005-05-31,NaN,2006-05-31,NaN
355,FA,2004-12-31,2004-12-31,NaN,2006-01-02,NaN
356,APLD,2004-05-31,2004-05-31,NaN,2005-05-31,NaN


In [43]:
close_df.to_csv("stock_prices.csv")